In [5]:
import pandas as pd

df = pd.read_csv("../data/past_qna_result.csv", encoding="utf-8")
df = df.drop("공연장", axis=1)
df = df.rename(columns={"대형마트": "마트", "한강": "강"})
df.head()

,label,PC방,경찰,공공체육시설,공원,다이소,마트,도서관,따릉이,주민센터,지하철,카페,노래방,편의점,산,하천,강,한의원,헬스장
0,신수동,75,25,25,50,50,50,25,50,75,25,50,50,75,25,75,100,25,50
1,효자동,25,25,25,25,50,50,25,25,25,25,75,25,0,25,25,25,0,0
2,청암동,25,25,25,25,50,50,25,25,25,25,0,25,50,25,100,100,0,0
3,염리동,25,25,75,50,50,50,100,50,75,25,50,50,75,25,25,75,50,50
4,용두동,50,75,50,75,50,100,100,75,50,100,25,50,75,25,75,25,50,50


In [6]:
df.columns

Index(['label', 'PC방', '경찰', '공공체육시설', '공원', '다이소', '마트', '도서관', '따릉이', '주민센터',
       '지하철', '카페', '노래방', '편의점', '산', '하천', '강', '한의원', '헬스장'],
      dtype='object')

In [8]:
from sqlalchemy import create_engine, select
from sqlalchemy.orm import sessionmaker
import os
import sys

sys.path.append("../")
from backend.models import (
    HomieHistory,
    HomieQnAHistory,
    User,
    HomieQuestion,
    HomieAnswer,
    HomieDong,
)

engine = create_engine(os.getenv("DB_URL"))
Session = sessionmaker(bind=engine)
session = Session()

user_id = session.query(User.id).filter(User.email == "alpaca9100@gmail.com").first()[0]

for data in df.itertuples():
    # homie_dong_id = session.query(HomieDong.id).filter(HomieDong.dong == data.label)
    homie_dong_id = session.scalar(
        select(HomieDong.id).where(HomieDong.dong == data.label).limit(1)
    )
    new_homie_history = HomieHistory(user_id=user_id, homie_dong_id=homie_dong_id)
    session.add(new_homie_history)
    session.flush()
    scores = {k: v for k, v in data._asdict().items() if k not in ["label", "Index"]}
    for key, val in scores.items():
        print(key, val)
        homie_question_id = (
            session.query(HomieQuestion.id)
            .filter(HomieQuestion.sub_category == key)
            .first()[0]
        )
        if key == "약국" and val not in [0, 50, 100]:
            if val == 25:
                val = 50
            elif val == 75:
                val = 100
        homie_answer_id = (
            session.query(HomieAnswer.id)
            .filter(
                HomieAnswer.homie_question_id == homie_question_id,
                HomieAnswer.score == val,
            )
            .first()[0]
        )

        new_homie_qna_history = HomieQnAHistory(
            homie_history_id=new_homie_history.id,
            homie_question_id=homie_question_id,
            homie_answer_id=homie_answer_id,
        )
        session.add(new_homie_qna_history)
session.commit()
session.close()

PC방 75
경찰 25
공공체육시설 25
공원 50
다이소 50
마트 50
도서관 25
따릉이 50
주민센터 75
지하철 25
카페 50
노래방 50
편의점 75
산 25
하천 75
강 100
한의원 25
헬스장 50
PC방 25
경찰 25
공공체육시설 25
공원 25
다이소 50
마트 50
도서관 25
따릉이 25
주민센터 25
지하철 25
카페 75
노래방 25
편의점 0
산 25
하천 25
강 25
한의원 0
헬스장 0
PC방 25
경찰 25
공공체육시설 25
공원 25
다이소 50
마트 50
도서관 25
따릉이 25
주민센터 25
지하철 25
카페 0
노래방 25
편의점 50
산 25
하천 100
강 100
한의원 0
헬스장 0
PC방 25
경찰 25
공공체육시설 75
공원 50
다이소 50
마트 50
도서관 100
따릉이 50
주민센터 75
지하철 25
카페 50
노래방 50
편의점 75
산 25
하천 25
강 75
한의원 50
헬스장 50
PC방 50
경찰 75
공공체육시설 50
공원 75
다이소 50
마트 100
도서관 100
따릉이 75
주민센터 50
지하철 100
카페 25
노래방 50
편의점 75
산 25
하천 75
강 25
한의원 50
헬스장 50
PC방 50
경찰 100
공공체육시설 25
공원 50
다이소 50
마트 50
도서관 25
따릉이 75
주민센터 25
지하철 25
카페 50
노래방 50
편의점 25
산 100
하천 25
강 25
한의원 25
헬스장 75
PC방 100
경찰 50
공공체육시설 50
공원 75
다이소 100
마트 50
도서관 75
따릉이 75
주민센터 75
지하철 75
카페 50
노래방 75
편의점 75
산 50
하천 25
강 75
한의원 75
헬스장 75
PC방 75
경찰 25
공공체육시설 25
공원 50
다이소 50
마트 50
도서관 25
따릉이 50
주민센터 75
지하철 25
카페 50
노래방 50
편의점 75
산 25
하천 75
강 100
한의원 25
헬스장 50
PC방 50
경찰 25
공공체육시설 50
공원 